## Preprocessing

In [51]:
# Import our dependencies
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

# Define the learning rate scheduler function
def schedule(epoch, lr):
    if epoch < 10:
        return lr
    else:
        return lr * tf.math.exp(-0.1)

#  Import and read the charity_data.csv.
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [52]:
# Drop irrelevant columns
application_df = application_df.drop(['EIN', 'NAME'], axis=1)

In [53]:
## Binning for APPLICATION_TYPE
cutoff_value = 10
application_type_counts = application_df['APPLICATION_TYPE'].value_counts()
application_types_to_replace = application_type_counts[application_type_counts < cutoff_value].index.tolist()
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app, "Other")

# Verify to ensure successful binning
application_df['APPLICATION_TYPE'].value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
T9         156
T13         66
T12         27
T2          16
Other       11
Name: APPLICATION_TYPE, dtype: int64

In [54]:
# Binning for CLASSIFICATION
cutoff_value = 100
classification_counts = application_df['CLASSIFICATION'].value_counts()
classifications_to_replace = classification_counts[classification_counts < cutoff_value].index.tolist()
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls, "Other")

# Check to make sure binning was successful
application_df['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
Other      669
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
Name: CLASSIFICATION, dtype: int64

In [55]:
# Convert categorical data into a numerical format using one-hot encoding.
# Use pd.get_dummies() to perform one-hot encoding
application_df = pd.get_dummies(application_df)

In [56]:
# Separate data into features (X) and target (y)
X = application_df.drop(columns=['IS_SUCCESSFUL'])
y = application_df['IS_SUCCESSFUL']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [61]:
# Create a StandardScaler instances
scaler = StandardScaler()
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

## Compile, Train and Evaluate the Model

In [99]:
# Define the model - deep neural net, specifying the number of input features and hidden nodes for each layer.
nn = tf.keras.models.Sequential()

# Model Creation
nn = tf.keras.models.Sequential()
nn.add(tf.keras.layers.Dense(units=64, activation='relu', input_dim=X_train.shape[1]))
nn.add(tf.keras.layers.Dense(units=32, activation='relu'))
nn.add(tf.keras.layers.Dense(units=16, activation='relu'))

# Modify the learning rate and optimizer
nn.compile(loss="binary_crossentropy", optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), metrics=["accuracy"])

# Add L2 regularization to the dense layer
nn.add(tf.keras.layers.Dense(units=64, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.01)))

# Output layer with a single node and 'sigmoid' activation function for binary classification
nn.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

# Check the structure of the model
nn.summary()

Model: "sequential_26"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_79 (Dense)            (None, 64)                3456      
                                                                 
 dense_80 (Dense)            (None, 32)                2080      
                                                                 
 dense_81 (Dense)            (None, 16)                528       
                                                                 
 dense_82 (Dense)            (None, 64)                1088      
                                                                 
 dense_83 (Dense)            (None, 1)                 65        
                                                                 
Total params: 7217 (28.19 KB)
Trainable params: 7217 (28.19 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [100]:
# Technique 1: Learning Rate Scheduling
# Implement a learning rate scheduler callback
def schedule(epoch, lr):
    if epoch < 10:
        return lr
    else:
        return lr * tf.math.exp(-0.1)

lr_scheduler = tf.keras.callbacks.LearningRateScheduler(schedule)

# 2: Regularization Techniques
# Add dropout layers and batch normalization
nn.add(tf.keras.layers.Dropout(0.5))  # Adjust the dropout rate as needed
nn.add(tf.keras.layers.BatchNormalization())

# Output layer with a single node and 'sigmoid' activation function for binary classification
nn.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

In [101]:
# Model Compilation
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [102]:
# 3: Early Stopping
# Implement early stopping callback
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10)

In [103]:
# Train the model with callbacks
model = nn.fit(X_train_scaled, y_train, epochs=100, validation_data=(X_test_scaled, y_test), callbacks=[lr_scheduler, early_stopping])

Epoch 1/100
804/804 [==============================] - 6s 5ms/step - loss: 0.6908 - accuracy: 0.5875 - val_loss: 0.6296 - val_accuracy: 0.7243 - lr: 0.0010
Epoch 2/100
804/804 [==============================] - 4s 5ms/step - loss: 0.6575 - accuracy: 0.5962 - val_loss: 0.6239 - val_accuracy: 0.7223 - lr: 0.0010
Epoch 3/100
804/804 [==============================] - 5s 6ms/step - loss: 0.6551 - accuracy: 0.5986 - val_loss: 0.6263 - val_accuracy: 0.7216 - lr: 0.0010
Epoch 4/100
804/804 [==============================] - 4s 4ms/step - loss: 0.6546 - accuracy: 0.5987 - val_loss: 0.6259 - val_accuracy: 0.7250 - lr: 0.0010
Epoch 5/100
804/804 [==============================] - 4s 5ms/step - loss: 0.6555 - accuracy: 0.6001 - val_loss: 0.6273 - val_accuracy: 0.7234 - lr: 0.0010
Epoch 6/100
804/804 [==============================] - 5s 6ms/step - loss: 0.6541 - accuracy: 0.5985 - val_loss: 0.6356 - val_accuracy: 0.7219 - lr: 0.0010
Epoch 7/100
804/804 [==============================] - 4s 5ms/st

In [104]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 1s - loss: 0.6244 - accuracy: 0.7255 - 552ms/epoch - 2ms/step
Loss: 0.624355673789978, Accuracy: 0.7254810333251953


In [105]:
# Export our model to HDF5 file
optimized_model = "AlphabetSoupCharity_Optimization.h5"
nn.save(optimized_model)

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
